# Check Github for more info:
https://github.com/Exmantion/Flickr-Image-Downloader/

This script is meant only for personal use, just to enjoy your eyes with beautiful wallpapers.

In [1]:
#run here first (shift+enter)
!ls

sample_data


# Allowed URLS

In [2]:
### accepted urls:
urlss= ''' [ 
https://www.flickr.com/photos/wernerkrause/52427331246/sizes/l/
https://www.flickr.com/photos/wernerkrause/52427331246/
https://www.flickr.com/photos/wernerkrause/52427331246
https://www.flickr.com/photos/steffe/52491777467/in/contacts/
]'''

# How-To

In [ ]:
# Run first cell ^
# execute Options cell below
# run below code 'The script' section
# on the left tab there is a folder icon, click it
# click refresh 
# Paste all urls into input_url.txt file and ctrl+s
# run 'The script' section again
# next cell contains option to pack all of the photos into zip for easier downloading

# Options

In [7]:
FlickrFileNames = True 
  # False means file names of each photo will be with numbers, as in Flickr database states
  # True means title of the photo from main page, useful for further debugging or to find author of the photo
Debugging = 2 #0,1(default),2
  # 0 means that the output of below cell will output only final info
  # 1 means info of each downlaoded photo will be progressively displayed
  # 2 means all debbuging will also be saved in txt file called debugging.txt
DownloadFiles = True
  # Download photos yes or no
  # Helpful in case of Debugging 

# The script

In [4]:
import requests
import re
from datetime import datetime
exported_file = 'exported_urls.txt'
input_urls = 'input_urls.txt'
if Debugging == 2:
  debugging_file = 'debugging.txt'
  !touch $debugging_file
  debug = open(debugging_file, 'w')
!touch $input_urls
!touch $exported_file
input_urls_var = open(input_urls, 'r')
outputted_file = open(exported_file, 'w')
output_folder_name = 'gathered'
!mkdir -p $output_folder_name
statisctics = [0,0] #[success, error]
nr_of_photo = 1
if FlickrFileNames:
  custom_file_names = set()
seen = set()#find duplicates
duplicates_counter = 0
links_count = 0
for line in input_urls_var:
  if line.strip() in seen:
    duplicates_counter += 1
  else:
    seen.add(line.strip())
    links_count += 1
if Debugging > 0 and duplicates_counter>0:
  print(f"{duplicates_counter} duplicates were found and removed, photos to download: {links_count}\n")
if Debugging == 2:
  debug.write(f"{duplicates_counter} duplicates were found and removed, photos to download: {links_count}\n\n")

for url in seen:
  url = url.replace('\n',"")
  if Debugging > 0:
    print(f"{nr_of_photo}: Gathering data for: {url}")
  if Debugging == 2:
    debug.write(f"{nr_of_photo}: Gathering data for: {url}\n")

  if url.rfind("sizes") != -1: #check if photo link is in change sizes site
    url = url[0:url.rfind("sizes")]
    if Debugging > 0:
      print("\tURL has been corrected")
    if Debugging == 2:
      debug.write(f"\tURL has been corrected\n")

  html = requests.get(url).text
  output = []
  searchstring ='params: {"photoModel":'
  for line in iter(html.splitlines()):
    if searchstring in line:
      output.append(line)
  try:
    output = output[0][13:]
  except IndexError:
    if Debugging > 0:
      print("\tPhoto could not be obtained, wrong address probably")
    if Debugging == 2:
      debug.write(f"\tPhoto could not be obtained, wrong address probably\n")
    statisctics[1] += 1
    continue
  if Debugging > 0:
    print("\tGathered source... continuing")
  if Debugging == 2:
    debug.write(f"\tGathered source... continuing\n")

  false = False
  true = True
  null = -1
  outputdict = dict(eval(output))["photoModel"]['sizes']
  all_sizes = list(outputdict.keys())
  raw_url = outputdict[all_sizes[-1]]['src']
  direct_url = 'https://' + raw_url.replace('\/', '/')[2:]
  if Debugging > 0:
    print(f"\tAll image sizes available: {all_sizes}")
    print(f"\tDirect url: {direct_url}, raw url for debugging: {raw_url}")
  if Debugging == 2:
    debug.write(f"\tAll image sizes available: {all_sizes}\n")
    debug.write(f"\tDirect url: {direct_url}, raw url for debugging: {raw_url}\n")

  if FlickrFileNames:
    try:
      titled_name = html[html.find('<title>'):html.find('</title>')].replace("<title>", '')
      # titled_name = titled_name[:titled_name.find(' |')].replace(' ','_').replace("'","").replace("(", '').replace(")",'').replace("-", "_")
      titled_name = titled_name[:titled_name.find(' |')]
      titled_name = re.sub('[^a-zA-Z0-9 \n\.]', '', titled_name).replace('.','').replace(' ','_')
      if titled_name[:8] == 'Untitled':
        titled_name = titled_name + "_" + str(datetime.now().strftime("%d%m%Y_%H%M%S%f"))
      titled_name = f'{titled_name}_{str(all_sizes[-1])}{direct_url[direct_url.rfind("."):] }'
      titled_name = re.sub(r'&#x\.\.;',r'_', titled_name) 
    except:
      titled_name = f'NoCuteFilenameFound.{direct_url[direct_url.rfind("."):]}'
      
    #check if photo title was a duplicate 
    if titled_name in custom_file_names:
      titled_name = titled_name[:titled_name.find(".")] + "_" + str(datetime.now().strftime("%d%m%Y_%H%M%S%f")) + titled_name[titled_name.find("."):]
      custom_file_names.add(titled_name)
    else:
      custom_file_names.add(titled_name)
    
    if Debugging == 0:
      if DownloadFiles:
        !wget $direct_url --output-document=$titled_name >> /dev/null 2>&1 
    elif Debugging == 1:
      print(f"\tCustom filename: {titled_name}")
      if DownloadFiles:
        !wget $direct_url --output-document=$titled_name >> /dev/null 2>&1 | printf "File saved with a cute custom filename!"
    elif Debugging == 2:
      print(f"\tCustom filename: {titled_name}")
      if DownloadFiles:
        !wget $direct_url --output-document=$titled_name >> /dev/null 2>&1 | printf "File saved with a cute custom filename!"
      debug.write(f"\tCustom filename: {titled_name}\n")
      debug.write("\tFile saved with a cute custom filename!\n") 
    if DownloadFiles:
      !mv $titled_name $output_folder_name/$titled_name  # --directory-prefix=$output_folder_name
  else:
    if Debugging ==0:
      if DownloadFiles:
        !wget $direct_url -P $output_folder_name >> /dev/null 2>&1
    elif Debugging == 1:
      if DownloadFiles:
        !wget $direct_url -P $output_folder_name >> /dev/null 2>&1 | printf "File saved!"
    elif Debugging == 2:
      if DownloadFiles:
        !wget $direct_url -P $output_folder_name >> /dev/null 2>&1 | printf "File saved!"
      debug.write("File saved!") 
  nr_of_photo += 1
  outputted_file.write(direct_url+'\n')
  statisctics[0] += 1
  print(f"\n")

try:
  if sum(statisctics) != links_count:
    print("some files were not downloaded")
  print(f"Total files requested: {links_count} ({sum(statisctics)}), Success: {statisctics[0]}, Errors: {statisctics[1]}\nSuccess percentage: {statisctics[0]/sum(statisctics)*100:.2f}%")
  if Debugging == 2:
    debug.write(f"Total files requested: {links_count} ({sum(statisctics)}), Success: {statisctics[0]}, Errors: {statisctics[1]}\nSuccess percentage: {statisctics[0]/sum(statisctics)*100:.2f}%")
except ZeroDivisionError:
  pass

input_urls_var.close()
outputted_file.close()
if Debugging == 2:
  debug.close()

# Create zip with all downloaded files

In [5]:
import shutil
archive_name = f'{output_folder_name + "_" + str(datetime.now().strftime("%d%m%Y_%H%M%S"))}'
!cp -f $exported_file $output_folder_name/$exported_file
if Debugging == 2:
  !cp -f $debugging_file $output_folder_name/$debugging_file
shutil.make_archive(archive_name, 'zip', output_folder_name)

'/content/gathered_01012023_175125.zip'

# Start all over again


In [6]:
!rm -r gathered 
!rm *.zip
!rm *.txt
!rm *.jpg
!rm *.png

rm: cannot remove '*.jpg': No such file or directory
rm: cannot remove '*.png': No such file or directory
